In [1]:
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import subprocess
import os.path
import subprocess
import datetime
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
import numpy as np
import scipy
import seaborn as sns

In [85]:
def clickOnAd(driver):
    try:
        res = driver.find_element_by_xpath("//*[contains(text(),'Skip Ad')]").click()        
    except:
        print("didn't find ad video ad insertion")
#The function receive a string indicate the requested video quality
def downloadVideo(video_quality,video_name, url, duration_of_the_video,root_path):
    t_time = time.strftime("%H_%M_%S")
    funcInFile = "video"

    if not os.path.exists(root_path):
        os.makedirs(root_path)

    video_path = root_path + video_name +"\\"
    if not os.path.exists(video_path):
        os.makedirs(video_path)

    quality_path =  video_path + "\\"
    if not os.path.exists(quality_path):
        os.makedirs(quality_path)

    filename = quality_path +"videoPcap_"+video_name + "_"  + funcInFile + t_time + ".pcap"
    tsharkOut  = open(filename, "wb")
    tsharkCall = ["C:\\Program Files\\Wireshark\\tshark.exe","-F", "pcap", "-f", "port 443", "-i", "Wi-Fi", "-w", filename] #port 56847 and 51820

    tsharkProc = subprocess.Popen(tsharkCall, stdout=tsharkOut, executable="C:\\Program Files\\Wireshark\\tshark.exe")
    chrome_options = webdriver.ChromeOptions()

    chrome_options.binary_location ="C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
    chrome_options.add_extension('C:\\Users\\92344\\Desktop\\Code\\adblock.crx')


    #chrome_options.add_extension('Ultrasurf.crx')

    driver = webdriver.Chrome(executable_path='C:\\Users\\92344\\Desktop\\Code\\chromedriver.exe',options=chrome_options)

    wait = WebDriverWait(driver, 20)
    driver.get(url)

    start_time = time.time()

    main_window = driver.current_window_handle

    #driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 'w')
    driver.switch_to_window(main_window)

    time.sleep(10)

    clickOnAd(driver)
    time.sleep(2)
    clickOnAd(driver)

    timersetting = False
    time.sleep(10)

    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Play']"))).click()
        end_time = time.time()
    except:
        timersetting = True
        end_time = time.time()

    time_elapsed = (end_time - start_time)

    if(timersetting):
        duration_of_the_video = duration_of_the_video - time_elapsed


    time.sleep(duration_of_the_video)
    driver.quit()
    tsharkProc.terminate()
def harvest_video(amount,name,url,duration,root_path):
    for x in range(0, amount):
        downloadVideo("Auto",name,url,duration,root_path)

def download_Pcaps(root_path,total_pcaps,pcaps_folder_name):
        folder_exist = False
        entries = os.listdir(root_path)
        if(len(entries)==0):
            os.makedirs(root_path+pcaps_folder_name)
        if(entries):
            for folder in entries:
                   if(folder==pcaps_folder_name):
                        folder_exist = True
            if(folder_exist):
                pcaps_in_folder = len(os.listdir(root_path+pcaps_folder_name))
                no_of_pcaps_downloading = total_pcaps - pcaps_in_folder
                if(no_of_pcaps_downloading > 0 ):
                        filename = pcaps_folder_name
                        urlF = "https://youtu.be/"+pcaps_folder_name
                        amount = no_of_pcaps_downloading 
                        link100_url = urlF
                        link100_duration = 120
                        harvest_video(amount,filename,link100_url,link100_duration,root_path)
            else:
                if not os.path.exists(root_path+pcaps_folder_name):
                    os.makedirs(root_path+pcaps_folder_name)
                    filename = pcaps_folder_name
                    urlF = "https://youtu.be/"+pcaps_folder_name
                    amount = total_pcaps
                    link100_url = urlF
                    link100_duration = 120
                    harvest_video(amount,filename,link100_url,link100_duration,root_path)
        else:
            print("Root Dir Empty")

#packet per sec
def read_pcap(filename, fields=[], display_filter="", timeseries=False, strict=False):
    if timeseries:
        fields = ["frame.time_epoch"] + fields
    fieldspec = " ".join("-e %s" % f for f in fields)
    display_filters = fields if strict else []
    if display_filter:
        display_filters.append(display_filter)
    filterspec = "-R '%s'" % " and ".join(f for f in display_filters)
    options = "-r %s -2 -T fields -Eheader=y" % filename
    cmd = "tshark %s %s" % (options, fieldspec)
    proc = subprocess.Popen(cmd, shell = True,stdout=subprocess.PIPE)
    df = pd.read_table(proc.stdout)
    return df

def _replaceitem(x):
    if x < 100000:
        return 0
    
    else:
        return x
def _replaceitem2(x):
    if x < 0:
        return 0
    
    else:
        return x
def packetPerSecond(base_path,pps_csv_file_path):
    with open(pps_csv_file_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file)
            dir_path= base_path
            fields=["ip.src", "ip.dst", "ip.proto","frame.len"]
            main_dir = os.listdir(dir_path)
            filelist=os.listdir(dir_path)
            for file in filelist:
                #file path isye
                print(file)
                temp=read_pcap(os.path.join(base_path, file), fields, timeseries=True, strict=True)
                temp["frame.time_epoch"]=pd.to_datetime(temp["frame.time_epoch"],unit='s')
                source_address=temp[temp["ip.dst"] == "192.168.1.170"]
                
                packet_per_second=source_address.resample("s", on='frame.time_epoch').count()
                print(packet_per_second)
                BPS_list=packet_per_second['frame.len'][0:120]
                BPS_list = list(map(_replaceitem2, BPS_list))
                lenBPS = len(BPS_list)
                if lenBPS < 120:
                    print("len is less")
                    diff = 120- lenBPS
                    for d in range(diff):
                        BPS_list.append(0)
                writer.writerow(BPS_list)
                print(BPS_list)
def bytePerSecond(base_path,bps_csv_file_path):
    with open(bps_csv_file_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file)
            dir_path= base_path
            fields=["ip.src", "ip.dst", "ip.proto","frame.len"]
            main_dir = os.listdir(dir_path)
            filelist=os.listdir(dir_path)
            for file in filelist:      
                temp=read_pcap(os.path.join(base_path, file), fields, timeseries=True, strict=True)
                temp["frame.time_epoch"]=pd.to_datetime(temp["frame.time_epoch"],unit='s')
                source_address=temp[temp["ip.dst"] == "192.168.1.170"]
                packet_per_second=source_address.resample("s", on='frame.time_epoch').sum()
                BPS_list=packet_per_second['frame.len'][0:120]
                BPS_list = list(map(_replaceitem, BPS_list))
                lenBPS = len(BPS_list)
                if lenBPS < 120:
                    print("len is less")
                    diff = 120- lenBPS
                    for d in range(diff):
                        BPS_list.append(0)
                writer.writerow(BPS_list)
                print("done")
def bytePerPeek(bps_csv_file_path,bpp_csv_file_path):
    xx = []
    yy = []
    sum=0;
    with open(bps_csv_file_path,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter = ',')
        for col in plots:
            col = col[0:-1]
            print(col)
            for i in range(len(col)):
                if i<len(col)-1:   
                    if int(col[i+1])!= 0 :
                        sum=sum+int(col[i])
                    else:
                            sum=sum+int(col[i])
                            if sum!=0:
                                yy.append(sum)
                                xx.append(i)
                                sum=0
                else:
                    sum=sum+int(col[i])
                    yy.append(sum)
                    xx.append(i)
    data = {'x': xx, 'y': yy}  
    df = pd.DataFrame(data)  
    df.to_csv(bpp_csv_file_path)

In [89]:
root_path="C:\\Users\\92344\\Desktop\\Code\\PCAPS/"
filename = "mAZ2uO9qu5M"
pps_csv_root_path = r"C:\Users\92344\Desktop\Code\PPS/"
bps_csv_root_path = r"C:\Users\92344\Desktop\Code\BPS/"
bpp_csv_root_path = r"C:\Users\92344\Desktop\Code\BPP/"

pps_csv_file_path = os.path.join(pps_csv_root_path, filename+".csv")
bps_csv_file_path = os.path.join(bps_csv_root_path, filename+".csv")
bpp_csv_file_path = os.path.join(bpp_csv_root_path, filename+".csv")

download_Pcaps(root_path,6,filename)
packetPerSecond(os.path.join(root_path, filename),pps_csv_file_path)
bytePerSecond(os.path.join(root_path, filename),bps_csv_file_path)
#bytePerPeek(bps_csv_file_path,bpp_csv_file_path)

C:\Users\92344\anaconda3\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: use driver.switch_to.window instead


didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
didn't find ad video ad insertion
videoPcap_mAZ2uO9qu5M_video11_34_39.pcap
                     frame.time_epoch  ip.src  ip.dst  ip.proto  frame.len
frame.time_epoch                                                          
2021-11-11 06:34:44                11      11      11        11         11
2021-11-11 06:34:45                19      19      19        19         19
2021-11-11 06:34:46                 7       7       7         7          7
2021-11-11 06:34:47                 7       7       7         7          7
2021-11-11 06:34:48                15      15      15        15         15
...                       